Ноутбук сделан для выполнения 1-ого задания тренировок по ML NLP

Задание - создание собственного Word2Vec

In [ ]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2025-09-20 05:47:06--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2025-09-20 05:47:06--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9107a7d1d4521df587ebd7d016.dl.dropboxusercontent.com/cd/0/inline/CxupLv2RMDJQdVZjI-DRNkMEODvQ1DdiVt2avcyolufRJBpiMI9C4dNTN7qTCnGob2I1lZz5HL4uq8w_UxFzyStB-B2KF2N-KM257jvsJ15TJur3MevdBX7nCVCMam-nPUE/file?dl=1# [following]
--2025-09-20 05:47:06--  https://uc9107a7d1d4521df587ebd7d016.dl.dropboxusercontent.com/cd/0/inline/CxupLv2RM

In [ ]:
data=list(open("./quora.txt",encoding="utf-8"))
data[49]

'Should cricket be our national sport and not hockey?\n'

Токенизация


In [ ]:
tokenizer=WordPunctTokenizer()

print(tokenizer.tokenize(data[49]))

['Should', 'cricket', 'be', 'our', 'national', 'sport', 'and', 'not', 'hockey', '?']


In [ ]:
data_tok=[
    tokenizer.tokenize(
      line.translate(str.maketrans("","",string.punctuation)).lower()
    )
    for line in data
]
print(data_tok[49])

['should', 'cricket', 'be', 'our', 'national', 'sport', 'and', 'not', 'hockey']


In [ ]:
data_tok=[x for x in data_tok if len(x)>=3]

проверки

In [ ]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

Ниже заданы константы ширины окна контекста и проведена предобработка для построения skip-gram модели.

In [ ]:
min_count =5
window_radius =5

In [ ]:
vocabulary_with_counter=Counter(chain.from_iterable(data_tok))

word_count_dict=dict()
for word, counter in vocabulary_with_counter.items():
  if counter>=min_count:
    word_count_dict[word]=counter

vocabulary =set(word_count_dict.keys())
del vocabulary_with_counter

In [ ]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {index: word for word, index in word_to_index.items()}

In [ ]:
context_pairs=[]

for text in data_tok:
  for i, central_word in enumerate(text):
    context_indicec=range(
        max(0,i-window_radius), min(i+window_radius,len(text))
    )
    for j in context_indicec:
      if j!=i:
        context_word=text[j]
        if central_word in vocabulary and context_word in vocabulary:
          context_pairs.append(
              (word_to_index[central_word],word_to_index[context_word])
          )
print (len(context_pairs))

40220313


SUBSAMPLING

In [ ]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """
    s=0
    for word in word_count_dict.keys():
      s+=word_count_dict[word]

    ### YOUR CODE HERE
    keep_prob_dict = {
        word: np.sqrt(threshold/(word_count_dict[word]/s)) for word in word_count_dict.keys()
    }  # THIS IS A PLACEHOLDER!
    return keep_prob_dict

Negative sampling

In [ ]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """
    Z=0
    for word in word_count_dict.keys():
      Z+=word_count_dict[word]**(3/4)

    ### YOUR CODE HERE
    negative_sampling_prob_dict = {
        word: word_count_dict[word]**(3/4)/Z for word in word_count_dict.keys()
    }  # THIS IS A PLACEHOLDER!
    return negative_sampling_prob_dict

Проверки

In [ ]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

In [ ]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

In [ ]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

In [ ]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
  batch = []
  neg_samples = []

  while len(batch) < batch_size:
    center, context =random.choice(context_pairs)
    if random.random()<keep_prob_array[center]:
      batch.append((center, context))
      neg_sample=np.random.choice(
          range(len(negative_sampling_prob_array)),
          size=num_negatives,
          p=negative_sampling_prob_array,
                                  )
      neg_samples.append(neg_sample)
  batch=np.array(batch)
  neg_samples = np.vstack(neg_samples)
  return batch, neg_samples

In [ ]:
batch_size=4
num_negatives=15
batch, neg_samples=generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

In [ ]:
print(batch.shape)

(4, 2)


In [ ]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, center_words, pos_context_words, neg_context_words):
        # center_words: [B], pos_context_words: [B], neg_context_words: [B, K]
        center = self.center_embeddings(center_words)                 # [B, D]
        pos    = self.context_embeddings(pos_context_words)           # [B, D]
        neg    = self.context_embeddings(neg_context_words)           # [B, K, D]

        pos_logits = (center * pos).sum(dim=1)                        # [B]
        # батчевый скалярный продукт для каждого из K негативов
        neg_logits = torch.bmm(neg, center.unsqueeze(2)).squeeze(2)   # [B, K]
        return pos_logits, neg_logits

In [ ]:
device = torch.device("cpu")

In [ ]:
vocab_size = len(word_to_index)
print(vocab_size)
embedding_dim = 32
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)
criterion = nn.BCEWithLogitsLoss()

28546


проверки

In [ ]:
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
print(params_counter, len(word_to_index) * embedding_dim * 2)
assert params_counter == len(word_to_index) * embedding_dim * 2

1826944 1826944


In [ ]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()
        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [25]:
steps=3001
batch_size=512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps
)

  0%|          | 0/3001 [00:00<?, ?it/s]

Step 0, Loss: 4.714033603668213, learning rate: [0.05]
Step 100, Loss: 3.5123893809318543, learning rate: [0.05]
Step 200, Loss: 2.5590882992744444, learning rate: [0.05]
Step 300, Loss: 2.283320007324219, learning rate: [0.05]
Step 400, Loss: 2.1210634875297547, learning rate: [0.05]
Step 500, Loss: 2.0456932640075682, learning rate: [0.05]
Step 600, Loss: 2.037844440937042, learning rate: [0.05]
Step 700, Loss: 2.0522425162792204, learning rate: [0.05]
Step 800, Loss: 2.080521545410156, learning rate: [0.05]
Step 900, Loss: 2.081893130540848, learning rate: [0.025]
Step 1000, Loss: 1.970003101825714, learning rate: [0.025]
Step 1100, Loss: 1.8710433161258697, learning rate: [0.025]
Step 1200, Loss: 1.8517334473133087, learning rate: [0.025]
Step 1300, Loss: 1.8062978196144104, learning rate: [0.025]
Step 1400, Loss: 1.7765582156181337, learning rate: [0.025]
Step 1500, Loss: 1.7563161444664002, learning rate: [0.025]
Step 1600, Loss: 1.719922926425934, learning rate: [0.0125]
Step 17

In [30]:
_model_parameters = model.parameters()
embedding_matrix_center = next(
    _model_parameters
).detach()  # Assuming that first matrix was for central word
embedding_matrix_context = next(
    _model_parameters
).detach()  # Assuming that second matrix was for context word

In [31]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

ещё проверки (на то что модель обучилась)

In [33]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
print(similarity_1,similarity_2)
assert similarity_1 > similarity_2

tensor([0.6512]) tensor([0.2809])


In [35]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [36]:
find_nearest("python", embedding_matrix_context, k=10)

[('burritos', 0.6027625203132629),
 ('designs', 0.6185270547866821),
 ('sdn', 0.6221513152122498),
 ('enders', 0.6270361542701721),
 ('java', 0.6276894211769104),
 ('dehradun', 0.6311379671096802),
 ('language', 0.65376877784729),
 ('reintroduced', 0.6819435358047485),
 ('assurance', 0.6846585869789124),
 ('python', 1.0000001192092896)]

Похоже на правду

In [37]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

In [38]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [39]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

In [40]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

In [41]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

figure(id='p1006', ...)

Сабмит в тестовую систему

In [42]:
# do not change the code in the block below
# __________start of block__________
import os
import json

assert os.path.exists(
    "words_subset.txt"
), "Please, download `words_subset.txt` and place it in the working directory"

with open("words_subset.txt") as iofile:
    selected_words = iofile.read().split("\n")


def get_matrix_for_selected_words(selected_words, embedding_matrix, word_to_index):
    word_vectors = []
    for word in selected_words:
        index = word_to_index.get(word, None)
        vector = [0.0] * embedding_matrix.shape[1]
        if index is not None:
            vector = embedding_matrix[index].numpy().tolist()
        word_vectors.append(vector)
    return word_vectors


word_vectors = get_matrix_for_selected_words(
    selected_words, embedding_matrix_context, word_to_index
)

with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
print("File saved to `submission_dict.json`")
# __________end of block__________

File saved to `submission_dict.json`
